### Import packages correction

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import tensorflow as tf

# Get your datasets

In [2]:
from data.dataset import H5PY_RW
from data.data_tools import read_metadata, males_keys, females_keys

H5_dic = read_metadata()
chunk_size = 512*10

males = H5PY_RW('test_raw_16k.h5py', subset = males_keys(H5_dic)).set_chunk(chunk_size).shuffle()
fem = H5PY_RW('test_raw_16k.h5py', subset = females_keys(H5_dic)).set_chunk(chunk_size).shuffle()
print 'Data with', len(H5_dic), 'male and female speakers'


We are on PC
Data with 40 male and female speakers


# Mixing the dataset

In [3]:
from data.dataset import Mixer

mixed_data = Mixer([males, fem], with_mask=False, with_inputs=True)

# Training set selection
mixed_data.select_split(0)

# Model pretrained loading

In [4]:
N = 256
max_pool = 128
batch_size = 8
learning_rate = 0.001

config_model = {}
config_model["type"] = "DPCL_train_front"

config_model["batch_size"] = batch_size
config_model["chunk_size"] = chunk_size

config_model["N"] = N
config_model["maxpool"] = max_pool
config_model["window"] = 1024

config_model["smooth_size"] = 20

config_model["alpha"] = learning_rate
config_model["reg"] = 1e-4
config_model["beta"] = 0.1
config_model["rho"] = 0.01

idd = ''.join('-{}={}-'.format(key, val) for key, val in sorted(config_model.items()))
config_model["type"] = "DPCL_finetuning"
config_model["batch_size"] = 1

In [5]:
from models.adapt import Adapt
import config
full_id = 'wispy-forest-8617'+idd

folder='DPCL_fine_tuning'
model = Adapt(config_model=config_model,pretraining= False, folder=folder)
path = os.path.join(config.workdir, 'log', 'DPCL_train_front')

from models.dpcl import DPCL

model.connect_front_back_to_separator(DPCL)

with model.graph.as_default():
    model.create_saver()
    model.restore_model(path, full_id)
    model.optimize
    model.tensorboard_init()



ID : AdaptiveNet-icy-mountain-2296
Tensor("separate/network/Gather:0", shape=(?, 2, ?), dtype=float32)
Tensor("separate/Reshape_5:0", shape=(?, ?, 256, 1), dtype=float32)
INFO:tensorflow:Restoring parameters from /home/anthony/das/log/DPCL_train_front/AdaptiveNet-wispy-forest-8617-N=256--alpha=0.001--batch_size=8--beta=0.1--chunk_size=5120--maxpool=128--reg=0.0001--rho=0.01--smooth_size=20--type=DPCL_train_front--window=1024-/model.ckpt


## Connect DPCL model to the front end and back end

In [ ]:
init = model.non_initialized_variables()

# Model creation

# Pretraining the model 

In [ ]:
nb_iterations = 1000
#initialize the model
model.sess.run(init)

for i in range(nb_iterations):
    X_in, X_mix, Ind = mixed_data.get_batch(batch_size)
    c = model.train(X_mix, X_in, learning_rate, i)
    print 'Step #'  ,i,' loss=', c 

    if i%20 == 0: #cost_valid < cost_valid_min:
        print 'DAS model saved at iteration number ', i,' with cost = ', c 
        model.save(i)